In [330]:
from mitiq.pec import NoisyOperation, NoisyBasis 
import pickle
from qiskit import QuantumCircuit
from qiskit.quantum_info import SuperOp, Kraus, Operator
import numpy as np
from mitiq.pec.representations.optimal import find_optimal_representation
from qiskit.circuit.library import RXGate, RZGate, XGate, YGate, ZGate, IGate
from itertools import product

np.set_printoptions(precision=3, suppress=True)

In [443]:
with open("XZ.kraus","rb") as f:
    krausops = pickle.load(f)

krausops.append(np.identity(2)) #I did not get idle in the tomography!
superops = [SuperOp(Kraus(k)) for k in krausops] #Convert to superoperators

#Create a list of gates matching the ideal ones
ideal_gates = [RXGate(-np.pi/2), RZGate(-np.pi/2)]

In [444]:
def get_gate_sequences(len):
    NoisyOperations = []
    for string in product(zip(superops,ideal_gates), repeat=len):
        qc = QuantumCircuit(1)
        superop = SuperOp(np.identity(4))
        for (noisy_op, ideal_op) in string:
            qc.append(ideal_op,[0])
            superop = superop.compose(noisy_op)
        NoisyOperations.append(NoisyOperation(qc, superop.data))
    
    return NoisyOperations.copy()

NoisyOperations = []
for i in range(3):
    NoisyOperations = np.concatenate((NoisyOperations, get_gate_sequences(i+1)))

basis = NoisyBasis(*NoisyOperations)

In [447]:
optimal_circuit = QuantumCircuit(1,1)
optimal_circuit.rx(-np.pi/2,0)

optimal_rep = find_optimal_representation(optimal_circuit, basis, tol=.01)
print(optimal_rep)

q_0: ───Rx(-0.5π)─── = 0.009*(q_0: ───Rz(-0.5π)───Rx(-0.5π)───)+0.017*(q_0: ───Rx(-0.5π)───Rz(-0.5π)───)+0.005*(q_0: ───Rx(-0.5π)───Rz(-0.5π)───Rx(-0.5π)───)-0.006*(q_0: ───Rz(-0.5π)───Rz(-0.5π)───Rz(-0.5π)───)-0.002*(q_0: ───Rz(-0.5π)───Rz(-0.5π)───)+0.991*(q_0: ───Rx(-0.5π)───)-0.004*(q_0: ───Rx(-0.5π)───Rz(-0.5π)───Rz(-0.5π)───)


In [448]:
print(optimal_rep.norm)

1.035708499454329


In [449]:
matrices = [m.channel_matrix for m in optimal_rep.noisy_operations]
mat = sum(m*coeff for (m,coeff) in zip(matrices, optimal_rep.coeffs))
print(mat)

[[ 0.506+0.j    -0.007-0.49j  -0.007+0.49j   0.502-0.j   ]
 [-0.003-0.492j  0.5  -0.01j   0.49 +0.007j  0.01 +0.49j ]
 [-0.003+0.492j  0.49 -0.007j  0.5  +0.01j   0.01 -0.49j ]
 [ 0.506-0.j     0.007+0.49j   0.007-0.49j   0.509+0.j   ]]


In [370]:
print(SuperOp(RXGate(np.pi/2)).data)

[[0.5+0.j  0. -0.5j 0. +0.5j 0.5+0.j ]
 [0. -0.5j 0.5+0.j  0.5+0.j  0. +0.5j]
 [0. +0.5j 0.5+0.j  0.5+0.j  0. -0.5j]
 [0.5+0.j  0. +0.5j 0. -0.5j 0.5+0.j ]]
